# Init

In [139]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

# Generator Answer

In [13]:
src_fname = 'stage_1-glm_4_plus-answer_generator-v1.0.0.json'
saved_fname = 'submit-stage_1-250125.json'
template_fname = 'submit_example.json'

src_fpath = os.path.join('answer_tmp', src_fname)
saved_fpath = os.path.join('answer', saved_fname)
template_fpath = os.path.join('data', template_fname)

In [14]:
raw_data = parse_data.read_json(src_fpath)
answers = parse_data.read_json(template_fpath)

for i in raw_data:
    tid = i['tid']
    try:
        answer = i['answer_generator']['stage_1']
        
        for j in answers:
            if j['tid'] == tid:
                j['team'][0]['answer'] = answer
    except:
        print(tid)

parse_data.write_json(answers, saved_fpath)

## Tokenizer

In [3]:
import hanlp

hanlp.pretrained.tok.ALL # 语种见名称最后一个字段或相应语料库
hanlp.pretrained.mtl.ALL
hanlp.pretrained.ner.ALL # 语种见名称最后一个字段或相应语料库

tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
# ner = hanlp.load(hanlp.pretrained.ner.MSRA_NER_ELECTRA_SMALL_ZH)

{'SIGHAN2005_PKU_CONVSEG': 'https://file.hankcs.com/hanlp/tok/sighan2005-pku-convseg_20200110_153722.zip',
 'SIGHAN2005_MSR_CONVSEG': 'https://file.hankcs.com/hanlp/tok/convseg-msr-nocrf-noembed_20200110_153524.zip',
 'CTB6_CONVSEG': 'https://file.hankcs.com/hanlp/tok/ctb6_convseg_nowe_nocrf_20200110_004046.zip',
 'PKU_NAME_MERGED_SIX_MONTHS_CONVSEG': 'https://file.hankcs.com/hanlp/tok/pku98_6m_conv_ngram_20200110_134736.zip',
 'LARGE_ALBERT_BASE': 'https://file.hankcs.com/hanlp/tok/large_corpus_cws_albert_base_20211228_160926.zip',
 'SIGHAN2005_PKU_BERT_BASE_ZH': 'https://file.hankcs.com/hanlp/tok/sighan2005_pku_bert_base_zh_20201231_141130.zip',
 'COARSE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/coarse_electra_small_20220616_012050.zip',
 'FINE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/fine_electra_small_20220615_231803.zip',
 'CTB9_TOK_ELECTRA_SMALL': 'https://file.hankcs.com/hanlp/tok/ctb9_electra_small_20220215_205427.zip',
 'CTB9_TOK_ELECTRA_BASE': 'http:/

{'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_small_20201223_035557.zip',
 'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_base_20201223_201906.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip',
 'CLOSE_TOK_POS_NER_SRL_UDEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20220626_175100.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_base_20210111_124519.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ERNIE_GRAM_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip',
 'KYOTO_EVAHAN_TOK_LEM_POS_UDEP_LZH': 'https

{'MSRA_NER_BERT_BASE_ZH': 'https://file.hankcs.com/hanlp/ner/ner_bert_base_msra_20211227_114712.zip',
 'MSRA_NER_ALBERT_BASE_ZH': 'https://file.hankcs.com/hanlp/ner/msra_ner_albert_base_20211228_173323.zip',
 'MSRA_NER_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/ner/msra_ner_electra_small_20220215_205503.zip',
 'CONLL03_NER_BERT_BASE_CASED_EN': 'https://file.hankcs.com/hanlp/ner/ner_conll03_bert_base_cased_en_20211227_121443.zip'}

In [15]:
tok.dict_force = {'JD.com, Inc.', '上海建工'}

print(tok(['600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？', '今天是2021年12月24日，创近半年新高的股票有几只？', 'TOUR他是否已经退市了？（是或者否）', '铜陵有色金属集团股份有限公司的法人代表是谁？', '海信视像科技股份有限公司在什么时候成立的，XXXX-XX-XX？', '截止2021-06-17上海建工的近一周成交金额（万元）是多少？', 'JD.com, Inc.这家公司在美股英文名称是什么？']))

[['600872', '的', '全称', '、', 'A股', '简称', '、', '法人', '、', '法律顾问', '、', '会计师', '事务所', '及', '董秘', '是', '？'], ['今天', '是', '2021年12月24日', '，', '创', '近', '半年', '新高', '的', '股票', '有', '几只', '？'], ['TOUR', '他', '是否', '已经', '退市', '了', '？', '（', '是', '或者', '否', '）'], ['铜陵有色金属集团股份有限公司', '的', '法人', '代表', '是', '谁', '？'], ['海信视像科技股份有限公司', '在', '什么', '时候', '成立', '的', '，', 'XXXX-XX-XX', '？'], ['截止', '2021-06-17', '上海建工', '的', '近', '一周', '成交', '金额', '（', '万元', '）', '是', '多少', '？'], ['JD.com, Inc.', '这', '家', '公司', '在', '美股', '英文', '名称', '是', '什么', '？']]


In [2]:
print(tok(['600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？', '今天是2021年12月24日，创近半年新高的股票有几只？', 'TOUR他是否已经退市了？（是或者否）', '铜陵有色金属集团股份有限公司的法人代表是谁？', '海信视像科技股份有限公司在什么时候成立的，XXXX-XX-XX？', '截止2021-06-17上海建工的近一周成交金额（万元）是多少？', 'JD.com, Inc.这家公司在美股英文名称是什么？'], tasks='ner*'))

[['600872', '的', '全称', '、', 'A股', '简称', '、', '法人', '、', '法律顾问', '、', '会计师', '事务所', '及', '董秘', '是', '？'], ['今天', '是', '2021年12月24日', '，', '创', '近', '半年', '新高', '的', '股票', '有', '几只', '？'], ['TOUR', '他', '是否', '已经', '退市', '了', '？', '（', '是', '或者', '否', '）'], ['铜陵有色金属集团股份有限公司', '的', '法人', '代表', '是', '谁', '？'], ['海信视像科技股份有限公司', '在', '什么', '时候', '成立', '的', '，', 'XXXX-XX-XX', '？'], ['截止', '2021-06-17', '上海', '建工', '的', '近', '一周', '成交', '金额', '（', '万元', '）', '是', '多少', '？'], ['JD.com', ',', 'Inc.', '这', '家', '公司', '在', '美股', '英文', '名称', '是', '什么', '？']]


In [8]:
tok.dict_force = {'上海建工'}

print(tok(['湘电股份的十二个月收盘最高价?']))

[['湘电股份', '的', '十二个月', '收盘', '最高', '价', '?']]


# Clean NER Result

In [142]:
question_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'glm_4_plus-market_classifier-v1.0.0.json')

questions = parse_data.read_json(question_path)
# sort the questions by tid
questions = sorted(questions, key=lambda x: int(x['tid'].split('-')[-1]))

In [143]:
for q in questions:
    t = q['ner']

    if not t:
        continue
    
    t = t['stage_1']['sql']

    for k, v in t.items():
        tmp = []
        for j in v:
            if j['result']:
                tmp.append(j['result'])

        if len(tmp) > 1:
            print(tmp)

[[{'ID': 16808232845300, 'InnerCode': 1000872, 'CompanyCode': 4057, 'SecuCode': '02628', 'ChiName': '中国人寿保险股份有限公司', 'ChiNameAbbr': None, 'EngName': 'China Life Insurance Company Limited', 'EngNameAbbr': 'CHINA LIFE', 'SecuAbbr': '中国人寿', 'ChiSpelling': 'ZGRS', 'SecuMarket': 72, 'SecuCategory': 3, 'ListedDate': '2003-12-18 12:00:00.000', 'ListedSector': 1, 'ListedState': 1, 'XGRQ': '2018-11-29 11:24:25.720', 'JSID': 596805865723, 'DelistingDate': None, 'ISIN': 'CNE1000002L3', 'FormerName': None, 'TradingUnit': 1000.0, 'TraCurrUnit': 1100, 'InsertTime': '2005-10-12 02:23:57.983'}], [{'ID': 694505747566, 'InnerCode': 7005304, 'SecuCode': 'LFC', 'SecuAbbr': '中国人寿', 'ChiSpelling': 'ZGRS', 'SecuCategory': 75, 'SecuMarket': 78, 'ListedSector': None, 'ListedDate': '2003-12-17 12:00:00.000', 'ListedState': 5, 'ISIN': None, 'CompanyCode': 4057, 'UpdateTime': '2022-12-18 10:06:10.113', 'JSID': 694708338161, 'DelistingDate': '2022-09-02 12:00:00.000', 'InsertTime': '2022-09-05 12:12:01.187', 'EngNa

# Extract Column Names

In [88]:
import os
import re

import pandas as pd

cwd = os.getcwd()

data_dir = os.path.join(cwd, "data/table-column")

files = [i for i in os.listdir(data_dir) if "with_table_name" in i]

column_names = set()

for fname in files[:]:
    fpath = os.path.join(data_dir, fname)

    # skip line 1
    df = pd.read_table(fpath, sep="|", skiprows=[1], engine='python')
    df = df.iloc[:, 1:-1]  # 去掉多余的边界列
    df.columns = [col.strip() for col in df.columns]  # 去掉列名的空格

    # extract column_description

    cols = list(df['column_description'])

    for i in range(len(cols)):
        # sub whitespace
        cols[i] = re.sub('\s', '', cols[i])
        # remove content in ()
        cols[i] = re.sub('\(.+\)', '', cols[i])
        # remove 1.1 like 
        cols[i] = re.sub('\d\.\d[.)]?', '', cols[i])
        cols[i] = re.sub('\d[.)]?', '', cols[i])
        # remove 一、二、 like 
        cols[i] = re.sub('^[一二三四五六七八九十]、', '', cols[i])
        # "#"
        cols[i] = re.sub('[#]+', '', cols[i])
        # "加:"
        cols[i] = re.sub('(加:)?(减:)?', '', cols[i])
        # remove suffix digit, such as 成交量1
        cols[i] = re.sub('\d$', '', cols[i])
        cols[i] = re.sub('其中：', '', cols[i])
        
        
    column_names.update(set(cols))
    
column_names.remove('')

with open('column_names.txt', 'w') as f:
    for i in column_names:
        tmp = f.write(i + '\n')

In [144]:
import hanlp

hanlp.pretrained.tok.ALL # 语种见名称最后一个字段或相应语料库

tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)

tok.dict_force = column_names

{'SIGHAN2005_PKU_CONVSEG': 'https://file.hankcs.com/hanlp/tok/sighan2005-pku-convseg_20200110_153722.zip',
 'SIGHAN2005_MSR_CONVSEG': 'https://file.hankcs.com/hanlp/tok/convseg-msr-nocrf-noembed_20200110_153524.zip',
 'CTB6_CONVSEG': 'https://file.hankcs.com/hanlp/tok/ctb6_convseg_nowe_nocrf_20200110_004046.zip',
 'PKU_NAME_MERGED_SIX_MONTHS_CONVSEG': 'https://file.hankcs.com/hanlp/tok/pku98_6m_conv_ngram_20200110_134736.zip',
 'LARGE_ALBERT_BASE': 'https://file.hankcs.com/hanlp/tok/large_corpus_cws_albert_base_20211228_160926.zip',
 'SIGHAN2005_PKU_BERT_BASE_ZH': 'https://file.hankcs.com/hanlp/tok/sighan2005_pku_bert_base_zh_20201231_141130.zip',
 'COARSE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/coarse_electra_small_20220616_012050.zip',
 'FINE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/fine_electra_small_20220615_231803.zip',
 'CTB9_TOK_ELECTRA_SMALL': 'https://file.hankcs.com/hanlp/tok/ctb9_electra_small_20220215_205427.zip',
 'CTB9_TOK_ELECTRA_BASE': 'http:/

In [147]:
for i in column_names:
    if '分红' in i:
        print(i)

历史累计分红次数
是否分红
历史累计分红总额
分红股本基数
分红对象
变更前是否分红
本年单位累计分红
分红实施公告日
本年累计分红次数
分红意向公布日
本年累计分红总额
分红派息股本基准日
变更前分红股本基数


In [151]:
def han_tok(query: str) -> str:

    stop_words = {'？', '?', '有', '的', '多少', '人', '（', '）', '是', '在', '、', '分别', '了', '，'}

    tok_res = tok(query)
    res = []

    for i in tok_res:
        if i not in stop_words:
            res.append(i)

    return res

han_tok('新科技纳入过多少个子类概念？')

['新', '科技', '纳入', '过', '个子', '类', '概念']

# 给 Table 添加示例

In [181]:
import os
import re
import json

import pandas as pd

cwd = os.getcwd()

data_dir = os.path.join(cwd, "data/table-column")

files = [i for i in os.listdir(data_dir) if "with_table_name" in i]

ins_fpath = os.path.join(cwd, 'data/database-table/database-with_instances.json')

with open(ins_fpath, 'r', encoding='utf-8') as f:
    content = json.load(f)

for fname in files[:]:
    fpath = os.path.join(data_dir, fname)
    table_name = fname.split('-')[0]

    # skip line 1
    df = pd.read_table(fpath, sep="|", skiprows=[1], engine='python')
    df = df.iloc[:, 1:-1]  # 去掉多余的边界列
    df = df.fillna('')
    df.columns = [col.strip() for col in df.columns]  # 去掉列名的空格
    df['column_name_cleaned'] = df['column_name'].str.strip()

    for i in content:
        if i['table_name_en'] == table_name:
            data = i
            break
    
    tmp = data['sql_instances'][0]
    tmp = {k: str(v) if v is not None else 'null' for k, v in tmp.items()}
    df['数据示例'] = df['column_name_cleaned'].map(tmp)
    df['数据示例'] = df['数据示例'].str[:30]
    # 删除中间清理列 column_name_cleaned
    df = df.drop(columns=['column_name_cleaned'])

    # 转换成 Markdown 格式
    markdown_table = df.to_markdown(index=False)

    # 去除多余的空格和横线
    markdown_table = '\n'.join(re.sub('  ', '', line) for line in markdown_table.splitlines() if line.strip())
    # 去除多余的 --
    markdown_table = re.sub('\|:-+\|:-+\|:-+\|:-+\|:-+\|:-+\|', '|---|---|---|---|---|---|', markdown_table)  

    saved_fpath = os.path.join(data_dir, f'{table_name}-with_instance.md')  
    with open(saved_fpath, 'w') as f:
        t = f.write(markdown_table)

# check results

In [190]:
import json

fpath = '/Users/dnhb/Desktop/GitHub/zhipu-2024/final_results.json'

with open(fpath, 'r') as f:
    content = json.load(f)

gold = [i for i in range(1, 101)]

for i in content:
    tmp = int(i['tid'].split('tttt----')[-1])
    if tmp in gold:
        gold.remove(tmp)

gold

[]

In [195]:
import json

fpath = '/Users/dnhb/Desktop/GitHub/zhipu-2024/final_results.json'

with open(fpath, 'r') as f:
    content = json.load(f)

content = sorted(content, key=lambda x: int(x['tid'].split('-')[-1]))

with open('/Users/dnhb/Desktop/GitHub/zhipu-2024/final_results_sorted.json', 'w', encoding='utf-8') as file:
    # Use json.dump to write the list of dictionaries as a JSON array
    json.dump(content, file, ensure_ascii=False, indent=4)

# 给 table_name 添加 dbname

In [217]:
import os
import re
import json

import pandas as pd

cwd = os.getcwd()

data_dir = os.path.join(cwd, "data/table-column")

files = [i for i in os.listdir(data_dir) if "with_instance" in i]

ins_fpath = os.path.join(cwd, 'data/database-table/database-with_instances.json')

with open(ins_fpath, 'r', encoding='utf-8') as f:
    content = json.load(f)

for fname in files[:]:
    fpath = os.path.join(data_dir, fname)
    table_name = fname.split('-')[0]

    for i in content:
        if i['table_name_en'] == table_name:
            db = i['database_name_en']


    # skip line 1
    df = pd.read_table(fpath, sep="|", skiprows=[1], engine='python')
    df = df.iloc[:, 1:-1]  # 去掉多余的边界列
    df = df.fillna('')
    df.columns = [col.strip() for col in df.columns]  # 去掉列名的空格
    df['table_name'] = f'{db}.{table_name}'

    # 转换成 Markdown 格式
    markdown_table = df.to_markdown(index=False)

    # 去除多余的空格和横线
    markdown_table = '\n'.join(re.sub('  ', '', line) for line in markdown_table.splitlines() if line.strip())
    # 去除多余的 --
    markdown_table = re.sub('\|:-+\|:-+\|:-+\|:-+\|:-+\|:-+\|', '|---|---|---|---|---|---|', markdown_table)  

    saved_fpath = os.path.join(data_dir, f'{table_name}-with_instance.md')  
    with open(saved_fpath, 'w') as f:
        t = f.write(markdown_table)